
Originally created by Chelsey McGowan-Yallop, SAMS-UHI (sa06cm@sams.ac.uk)

Modified by Muriel Dunn for AFKABAN

This script uses model-predicted TS(f) spectra to train a machine learning
classifier, performs nested cross-validation, applies the classifier to
measured TS(f) spectra and outputs results files.

To use a different classifier, see the list of supported classifiers at:
https://github.com/hyperopt/hyperopt-sklearn and set as clf.

Sometimes the initial hyperparameter configuration selected by HyperOpt in each
split in the outer loop will be unsuccessful and all trials will fail. The
retry decorator forces it to try again until retry_limit is reached.

OUTPUT FILES:
    _NestedCV.pkl contains results of nested cross-validation procedure
    _Predictions.pkl contains measured TS(f) spectra with predicted labels
    _BestParams.pkl contains the optimal hyperparameters for the model
"""

In [13]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler
import lightgbm
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno  


# Read the dataframes

In [2]:
# Set paths
path = 'F:/AFKABAN/'
classifypath = 'F:/AFKABAN/Classify/'

In [3]:
# Read pickle to open on Stokes
ppath = f'{path}Pcod_190123/select/'
p2path = f'{path}Pcod2_240123/select/'
apath = f'{path}Acod_200123/select/'
pbpath = f'{path}Pandalus_260123/select/'
kpath = f'{path}Krill_270123/'

In [4]:
a_df_120 = pd.read_feather(apath+'a_df_120.feather')
a_df_200 = pd.read_feather(apath+'a_df_200.feather')

p_df_120 = pd.read_feather(ppath+'p_df_120.feather')
p_df_200 = pd.read_feather(ppath+'p_df_200.feather')

p2_df_120 = pd.read_feather(p2path+'p2_df_120.feather')
p2_df_200 = pd.read_feather(p2path+'p2_df_200.feather')

pb_df_120 = pd.read_feather(pbpath+'pb_df_120.feather')
pb_df_200 = pd.read_feather(pbpath+'pb_df_200.feather')

## Organize the data into a single df

In [5]:
df_120 = pd.concat([a_df_120,p_df_120,pb_df_120])
df_120 = df_120.reset_index(drop=True)


df_200 = pd.concat([a_df_200,p_df_200,pb_df_200])
df_200 = df_200.reset_index(drop=True)



In [6]:
df_120.to_feather(f'{classifypath}/df_120.feather')
df_200.to_feather(f'{classifypath}/df_200.feather')

# Run different iterations
### No preprocessing

In [7]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/AFKABAN/'
classifypath = 'F:/AFKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
preprocessing = [] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

In [17]:
#@retry(stop=stop_after_attempt(10))
def nested_cv(X, y, model, n_splits, n_folds, unique_id, le, path):
    
    """
    This function performs nested cross-validation with Bayesian hyperparameter
    optimisation. It uses stratified k-fold cross-validation in both the inner
    and outer loops. After each outer loop, it outputs the results to a .pkl
    file. As there is an element of randomness to the optimisation procedure,
    sometimes all trials will fail. If you re-run the script, it will import
    the incomplete .pkl file and try again.
    
    Note that this is a modified version that uses F1 score as the evaluation
    metric. It also calculates class-specific F1 scores and confusion matrices,
    which are added to the output dataframe.
    
    PARAMETERS:
        X: data minus labels
        y: labels
        model: HyperoptEstimator object
        n_splits: # of splits to use in outer K-fold cross-validation
        n_folds: # of folds to use in inner K-fold cross-validation
        unique_id: Unique name string for file output path
    Written by Chelsey McGowan-Yallop
    """
    cv = StratifiedKFold(n_splits=n_splits,
                         shuffle=True,
                         random_state=42) # Outer CV
    
    i_start = 0
    i_list = []
    results_df = None
    cv_path = path + unique_id + '_NestedCV.pkl'
        
    if os.path.isfile(cv_path) == True: # If CV is incomplete, resume
        results_df = pd.read_pickle(cv_path)
        i_start = results_df.Outer_fold.max() + 1
        print('Resuming cross-validation from fold ' + str(i_start + 1))
        
    # Generate indices to split data by StratifiedKFold
    # Append indices for each fold to list    
    for tr_i, te_i in cv.split(X,y):
        i_list.append([tr_i, te_i])
    
    # For each fold...
    for i in range(i_start, len(i_list)):
        results_list = []
        print('Beginning fold ' + str(i+1) + ' of ' + str(len(i_list)))
        
        # Split data into training and test tests
        X_train = X[i_list[i][0]]
        y_train = y[i_list[i][0]]
        X_test = X[i_list[i][1]]
        y_test = y[i_list[i][1]]

        start = time.time()
        
        # Fit the HyperoptEstimator to training data (optimise model)
        model.fit(X_train,
                  y_train,
                  n_folds=n_folds, # Inner stratified k-fold CV
                  cv_shuffle=True)
        
        end = time.time()
        duration = end - start

        # Use optimised model to predict labels for test data
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred, average='weighted') # Evaluate
        
        # Everything below: formats and/or calculates results for output file
        sorted_labels = np.sort(np.unique(y_test))
        unweighted_score = f1_score(y_test, y_pred,
                                    average=None,
                                    labels=sorted_labels)
        c_matrix = confusion_matrix(y_test, y_pred,
                                    labels=sorted_labels)

        for trial in range(len(model.trials.trials)):
                if model.trials.trials[trial].get('result').get('status') == 'ok':
                    trial_loss = model.trials.trials[trial].get('result').get('loss')
                    trial_duration = model.trials.trials[trial].get('result').get('duration')
                else:
                    trial_loss = np.nan
                    trial_duration = np.nan
            
                results_list.append([i,
                                     score,
                                     unweighted_score,
                                     le.inverse_transform(sorted_labels),
                                     c_matrix,
                                     duration,
                                     trial,
                                     trial_loss,
                                     trial_duration])
        
        append_df = pd.DataFrame(results_list,
                                 columns=['Outer_fold',
                                          'Outer_score',
                                          'Outer_unweighted_scores',
                                          'Outer_unweighted_score_labels',
                                          'Outer_confusion_matrix',
                                          'Outer_training_duration',
                                          'Trial',
                                          'Trial_loss',
                                          'Trial_duration'])
        if i == i_start:
            if results_df is not None:
                final_df = pd.concat([results_df,
                                      append_df],
                                      ignore_index=True)
            else:
                final_df = append_df
            final_df.to_pickle(cv_path)
        
        else:
            results_df = pd.read_pickle(cv_path)
            final_df = pd.concat([results_df,
                                  append_df],
                                  ignore_index=True)
            final_df.to_pickle(cv_path)
    return
        
            
def f1_loss(y_true, y_pred):
    """
    Custom loss function for HyperOpt.
    Uses F1 score instead of accuracy score, as the latter is inappropriate
    for multi-class classification.
    """
    return 1.0 - f1_score(y_true, y_pred, average='weighted')

def main_classify(df, clf, unique_id,path, preprocessing=[],  ex_preprocessing=[], timeout=300, n_jobs=-1, max_evals=50, n_splits = 10,
n_folds = 10):
    """
    Function to run nested cross validation then apply fit to whole dataset
    Uses F1 score instead of accuracy score, as the latter is inappropriate
    for multi-class classification.
    """
    
    # -- WRANGLE DATA ---------------------------------------------------------
    df_np = df.to_numpy()
    le = LabelEncoder() # Maps labels -> int (e.g. Atlantic cod -> 0, Polar cod -> 1)
    df['Species_le'] = le.fit_transform(df.Species)
    X = df_np[:,:-2] # Features, TS(f) only
    y = df['Species_le'].to_numpy() # Labels


    # -- NESTED CROSS-VALIDATION ----------------------------------------------

    model = HyperoptEstimator(classifier = clf,
                              preprocessing = preprocessing,
                              ex_preprocs = ex_preprocessing,
                              algo = tpe.suggest,
                              trial_timeout = timeout,
                              loss_fn = f1_loss,
                              max_evals = max_evals,
                              n_jobs = n_jobs)
    model

    nested_cv(X, y, model, n_splits, n_folds, unique_id, le, path)

    # -- RETRAIN MODEL --------------------------------------------------------

    print('Retraining model on full dataset')

    model = HyperoptEstimator(classifier = clf,
                              preprocessing = preprocessing,
                              ex_preprocs = ex_preprocessing,
                              algo = tpe.suggest,
                              trial_timeout = timeout,
                              loss_fn = f1_loss,
                              max_evals = max_evals,
                              n_jobs = n_jobs)

    model.fit(X, y, n_folds=n_folds, cv_shuffle=True)

    with open(path + unique_id + '_BestParams.pkl', 'wb') as handle:
        pickle.dump(model.best_model(), handle)
    return

def print_F1_mean(classifier):
    cv_df = classifier['cv_df']
    
    print('Mean class-weighted F1 score for : '
          + str(round(cv_df.Outer_score.mean(), 2))
          + ' ± '
          + str(round(cv_df.Outer_score.std(), 2)))

    mean_class_scores = np.mean(np.vstack(cv_df.Outer_unweighted_scores.values), axis=0)
    std_class_scores = np.std(np.vstack(cv_df.Outer_unweighted_scores.values), axis=0)

    for i, species in enumerate(cv_df.Outer_unweighted_score_labels.values[0]):
        print('Mean F1 score for ' + species + ': '
             + str(round(mean_class_scores[i], 2))
             + ' ± '
             + str(round(std_class_scores[i], 2)))
        
def read_results(unique_id ,pathh):
    ' read the classifier results and predictions from the selected classifier'
    
    main_path = f'{classifypath}/{unique_id}'
    cv_path = '_NestedCV.pkl'
    best_params = '_BestParams.pkl'

    # Load dataframes
    cv_df = pd.read_pickle(main_path + cv_path) # Nested CV results
    best_params = pd.read_pickle(main_path + best_params)

    return {'name':classifier, 'cv_df':cv_df, 'best_params':best_params}

#### kNN

In [19]:
# CLASSIFIER kNN 120
unique_id = 'kNN_test' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
main_classify(df_120, clf, unique_id, classifypath)

Beginning fold 1 of 10
100%|███████████████████████████████████████████████████████████████████| 1/1 [05:07<00:00, 307.48s/trial, best loss=?]


AllTrialsFailed: 

In [251]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier

main_classify(X_200, y_200, clf, unique_id, [])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.79s/trial, best loss: 0.11323328785811737]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.10s/trial, best loss: 0.09413369713506137]
Beginning fold 3 of 10
100%|█████████████████████████████████████████████████| 35/35 [00:03<00:00,  3.71s/trial, best loss: 0.096862210095498]


100%|█████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.95s/trial, best loss: 0.096862210095498]
Beginning fold 4 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.15s/trial, best loss: 0.1064120054570259]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.85s/trial, best loss: 0.11186903137789905]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:05<00:00,  5.96s/trial, best loss: 0.10081743869209814]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.04s/trial, best loss: 0.10081743869209814]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.96s/trial, best loss: 0.10899182561307907]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.33s/trial, best loss: 0.1171662125340599]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:05<00:00,  5.11s/trial, best loss: 0.10899182561307907]


100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.08s/trial, best loss: 0.0953678474114441]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.05s/trial, best loss: 0.10217983651226159]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:03<00:00,  3.73s/trial, best loss: 0.12147239263803677]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.02s/trial, best loss: 0.12147239263803677]


#### lightGBM
Takes 10x longer than kNN

In [252]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
main_classify(X_120, y_120, clf, unique_id, [])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.58s/trial, best loss: 0.04456824512534818]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:27<00:00, 27.13s/trial, best loss: 0.04317548746518107]
Beginning fold 3 of 10
100%|████████████████████████████████████████████████| 35/35 [00:28<00:00, 28.68s/trial, best loss: 0.0473537604456824]


100%|████████████████████████████████████████████████| 50/50 [00:16<00:00, 16.52s/trial, best loss: 0.0473537604456824]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:25<00:00, 25.16s/trial, best loss: 0.04178272980501396]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:44<00:00, 44.53s/trial, best loss: 0.04317548746518107]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:28<00:00, 28.53s/trial, best loss: 0.04317548746518107]


100%|███████████████████████████████████████████████| 50/50 [00:48<00:00, 48.86s/trial, best loss: 0.04038997214484674]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:55<00:00, 55.60s/trial, best loss: 0.04874651810584962]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:27<00:00, 27.56s/trial, best loss: 0.0473537604456824]
Beginning fold 9 of 10
100%|████████████████████████████████████████████████| 5/5 [02:14<00:00, 134.19s/trial, best loss: 0.05563282336578579]


100%|███████████████████████████████████████████████| 50/50 [00:34<00:00, 34.19s/trial, best loss: 0.05006954102920724]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:10<00:00, 10.67s/trial, best loss: 0.03894297635605004]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:51<00:00, 51.47s/trial, best loss: 0.04260651629072687]


100%|███████████████████████████████████████████████| 50/50 [00:28<00:00, 28.36s/trial, best loss: 0.04260651629072687]


In [253]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier

main_classify(X_200, y_200, clf, unique_id, [])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [01:19<00:00, 79.59s/trial, best loss: 0.06002728512960431]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:36<00:00, 36.65s/trial, best loss: 0.05320600272851295]
Beginning fold 3 of 10
100%|██████████████████████████████████████████████| 35/35 [02:40<00:00, 160.97s/trial, best loss: 0.06548431105047747]


100%|███████████████████████████████████████████████| 50/50 [00:22<00:00, 22.40s/trial, best loss: 0.06548431105047747]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [01:26<00:00, 86.32s/trial, best loss: 0.05729877216916779]
Beginning fold 5 of 10
100%|██████████████████████████████████████████████| 50/50 [00:24<00:00, 24.55s/trial, best loss: 0.054570259208731264]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:34<00:00, 34.97s/trial, best loss: 0.06948228882833785]


100%|███████████████████████████████████████████████| 50/50 [01:52<00:00, 112.45s/trial, best loss: 0.0681198910081744]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:32<00:00, 32.65s/trial, best loss: 0.05994550408719346]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:33<00:00, 33.37s/trial, best loss: 0.05585831062670299]
Beginning fold 9 of 10
100%|██████████████████████████████████████████████████| 5/5 [00:13<00:00, 13.14s/trial, best loss: 0.1485013623978202]


100%|███████████████████████████████████████████████| 50/50 [00:25<00:00, 25.05s/trial, best loss: 0.05722070844686644]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:27<00:00, 27.51s/trial, best loss: 0.05722070844686644]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:19<00:00, 19.69s/trial, best loss: 0.05153374233128838]


100%|███████████████████████████████████████████████| 50/50 [00:42<00:00, 42.48s/trial, best loss: 0.05153374233128838]


#### Support vector machine

In [263]:
# CLASSIFIER SVC 120
unique_id = 'SVC_02-05-2023_120' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
main_classify(X_120, y_120, clf, unique_id, [])

Resuming cross-validation from fold 7
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.76s/trial, best loss: 0.04178272980501396]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:59<00:00, 59.29s/trial, best loss: 0.04178272980501396]
Beginning fold 9 of 10
100%|███████████████████████████████████████████████| 35/35 [00:05<00:00,  5.86s/trial, best loss: 0.04728789986091797]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.94s/trial, best loss: 0.04728789986091797]
Beginning fold 10 of 10
 96%|███████████████████████████████████████████████████████████████████████   | 24/25 [00:00<?, ?trial/s, best loss=?]

job exception: C <= 0



 96%|███████████████████████████████████████████████████████████████████████   | 24/25 [00:04<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 10
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.23s/trial, best loss: 0.04867872044506261]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 50/50 [00:12<00:00, 12.75s/trial, best loss: 0.045112781954887216]


In [260]:
# CLASSIFIER SVC 200
unique_id = 'SVC_'+ d1 +'_200' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier

main_classify(X_200, y_200, clf, unique_id, [])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.61s/trial, best loss: 0.05729877216916779]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.01s/trial, best loss: 0.06002728512960431]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.90s/trial, best loss: 0.04502046384720326]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.00s/trial, best loss: 0.04502046384720326]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.36s/trial, best loss: 0.04638472032742158]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:07<00:00,  7.18s/trial, best loss: 0.04229195088676674]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:03<00:00,  3.96s/trial, best loss: 0.08991825613079019]


100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.78s/trial, best loss: 0.06130790190735691]
Beginning fold 7 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.13s/trial, best loss: 0.0681198910081744]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:11<00:00, 11.87s/trial, best loss: 0.0463215258855586]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.16s/trial, best loss: 0.08174386920980925]


100%|█████████████████████████████████████████████| 50/50 [05:03<00:00, 303.61s/trial, best loss: 0.053133514986375974]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.92s/trial, best loss: 0.05177111716621252]
Retraining model on full dataset
100%|██████████████████████████████████████████████| 41/41 [00:05<00:00,  5.28s/trial, best loss: 0.053987730061349715]


100%|██████████████████████████████████████████████| 50/50 [00:03<00:00,  3.93s/trial, best loss: 0.053987730061349715]


#### Gaussian Naive Bayes
Results were worst than kNN, LightGBM and SVC. Will not repeat.

In [264]:
# CLASSIFIER gNB 120
unique_id = 'gNB_'+ d1 +'_120' # Unique ID for output file paths
clf = gaussian_nb(unique_id)  # Classifier
main_classify(X_120, y_120, clf, unique_id, [])

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.55s/trial, best loss: 0.11699164345403901]
Beginning fold 2 of 10
100%|████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.02s/trial, best loss: 0.1058495821727019]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.89s/trial, best loss: 0.12116991643454034]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.26s/trial, best loss: 0.12116991643454034]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.12s/trial, best loss: 0.11281337047353757]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.96s/trial, best loss: 0.12116991643454034]
Beginning fold 6 of 10
100%|████████████████████████████████████████████████| 20/20 [00:05<00:00,  5.46s/trial, best loss: 0.1155988857938719]


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.57s/trial, best loss: 0.1155988857938719]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.38s/trial, best loss: 0.11420612813370479]
Beginning fold 8 of 10
100%|████████████████████████████████████████████████| 50/50 [00:03<00:00,  3.59s/trial, best loss: 0.1155988857938719]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:03<00:00,  3.56s/trial, best loss: 0.12100139082058414]


100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.95s/trial, best loss: 0.12100139082058414]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.20s/trial, best loss: 0.10709318497913767]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:04<00:00,  4.03s/trial, best loss: 0.11654135338345861]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.23s/trial, best loss: 0.11654135338345861]


In [265]:
# CLASSIFIER gNB 200
unique_id = 'gNB_'+ d1 +'_200' # Unique ID for output file paths
clf = gaussian_nb(unique_id)  # Classifier

main_classify(X_200, y_200, clf, unique_id, [])

Beginning fold 1 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.07s/trial, best loss: 0.2237380627557981]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.54s/trial, best loss: 0.22510231923601642]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.09s/trial, best loss: 0.21828103683492495]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.53s/trial, best loss: 0.21828103683492495]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.94s/trial, best loss: 0.22919508867667127]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.67s/trial, best loss: 0.22646657571623463]
Beginning fold 6 of 10
100%|████████████████████████████████████████████████| 20/20 [00:03<00:00,  3.59s/trial, best loss: 0.2316076294277929]


100%|████████████████████████████████████████████████| 50/50 [00:03<00:00,  3.56s/trial, best loss: 0.2316076294277929]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.55s/trial, best loss: 0.21798365122615804]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.08s/trial, best loss: 0.22479564032697552]
Beginning fold 9 of 10
100%|██████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.52s/trial, best loss: 0.2220708446866485]


100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.16s/trial, best loss: 0.2220708446866485]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.05s/trial, best loss: 0.22070844686648505]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:04<00:00,  4.07s/trial, best loss: 0.22699386503067487]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.14s/trial, best loss: 0.22699386503067487]


## Preprocessing
Any! 

In [7]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/AFKABAN/'
classifypath = 'F:/AFKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [8]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_120_anyPP' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify(df_120, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.13s/trial, best loss: 0.06267409470752094]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.95s/trial, best loss: 0.04317548746518107]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:07<00:00,  7.41s/trial, best loss: 0.06128133704735372]


100%|███████████████████████████████████████████████| 50/50 [00:03<00:00,  3.91s/trial, best loss: 0.06128133704735372]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.23s/trial, best loss: 0.06128133704735372]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.45s/trial, best loss: 0.05988857938718661]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:07<00:00,  7.07s/trial, best loss: 0.07103064066852371]


100%|██████████████████████████████████████████████| 50/50 [00:05<00:00,  5.34s/trial, best loss: 0.058495821727019504]
Beginning fold 7 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.58s/trial, best loss: 0.06685236768802227]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.17s/trial, best loss: 0.07520891364902504]
Beginning fold 9 of 10
100%|█████████████████████████████████████████████████| 5/5 [00:04<00:00,  4.41s/trial, best loss: 0.07510431154381081]


100%|████████████████████████████████████████████████| 50/50 [00:05<00:00,  5.84s/trial, best loss: 0.0598052851182197]
Beginning fold 10 of 10
100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.96s/trial, best loss: 0.05563282336578579]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:04<00:00,  4.08s/trial, best loss: 0.06015037593984962]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.70s/trial, best loss: 0.06015037593984962]


In [10]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_anyPP' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier

AZKABANML.main_classify(df_200, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.51s/trial, best loss: 0.07366984993178716]
Beginning fold 2 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.95s/trial, best loss: 0.08049113233287863]
Beginning fold 3 of 10
100%|███████████████████████████████████████████████| 35/35 [00:04<00:00,  4.98s/trial, best loss: 0.09413369713506137]


100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.35s/trial, best loss: 0.09413369713506137]
Beginning fold 4 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.15s/trial, best loss: 0.07366984993178716]
Beginning fold 5 of 10
100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.43s/trial, best loss: 0.05320600272851295]
Beginning fold 6 of 10
100%|███████████████████████████████████████████████| 20/20 [00:05<00:00,  5.35s/trial, best loss: 0.08991825613079019]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.17s/trial, best loss: 0.05177111716621252]
Beginning fold 7 of 10
100%|██████████████████████████████████████████████| 50/50 [00:06<00:00,  6.87s/trial, best loss: 0.049046321525885506]
Beginning fold 8 of 10
100%|███████████████████████████████████████████████| 50/50 [00:06<00:00,  6.04s/trial, best loss: 0.10217983651226159]
Beginning fold 9 of 10
100%|██████████████████████████████████████████████████| 5/5 [00:05<00:00,  5.16s/trial, best loss: 0.1049046321525886]


100%|███████████████████████████████████████████████| 50/50 [00:04<00:00,  4.30s/trial, best loss: 0.06948228882833785]
Beginning fold 10 of 10
100%|████████████████████████████████████████████████| 50/50 [00:04<00:00,  4.47s/trial, best loss: 0.0926430517711172]
Retraining model on full dataset
100%|███████████████████████████████████████████████| 41/41 [00:05<00:00,  5.96s/trial, best loss: 0.09447852760736197]


100%|███████████████████████████████████████████████| 50/50 [00:05<00:00,  5.86s/trial, best loss: 0.09447852760736197]


In [ ]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_anyPP' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_120, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
 98%|████████████████████████████████████████████████████████████████████████▌ | 49/50 [00:00<?, ?trial/s, best loss=?]

In [ ]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_anyPP' # Unique ID for output file paths
clf = lightgbm_classification(unique_id)  # Classifier
AZKABANML.main_classify(df_200, clf, unique_id, classifypath, preprocessing=any_preprocessing(unique_id))